### 0. Load environment variables
#### Case 1) If you use OpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'model_name' = XXXXXXXXXXXXXXX

##### Case 2) If you use AzureOpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_TYPE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_VERSION' = XXXXXXXXXXXXXXX
- 'OPENAI_API_BASE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'deployment_name' = XXXXXXXXXXXXXXX

In [1]:
import os
import sys  
from dotenv import load_dotenv
sys.path.extend(["..", '../..'])  
_ = load_dotenv('.env')


model_name = os.getenv('deployment_name') if os.getenv('OPENAI_API_TYPE') == 'azure' else os.getenv('model_name')
if os.getenv('OPENAI_API_TYPE') == 'azure':
    deployment_name = model_name

### 1. Fetch the prompt and create a chain (using LCEL)

In [2]:
from langchain import hub
prompt = hub.pull("jet-taekyo-lee/agent-scratchpad-compression")

from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
llm = AzureChatOpenAI(deployment_name=deployment_name, temperature=0) if os.getenv('OPENAI_API_TYPE') == 'azure' else ChatOpenAI(model_name=model_name, temperature=0)

agent_scratchpad_compression_chain = prompt | llm


### 2. Create a pseudo AgentAction log and observation for demonstration

In [29]:
# from utils.agent_components.configurations import Configurations
# from utils.agent_tools.tools.web_search_tools import get_web_search_tools
# from utils.agent_tools.tools.weather_tools import GetFromOpenWeatherMap
# from utils.agent_tools.tools.time_tools import GetFromDatetimeModule
# from utils.agent_tools.tools.python_repl_tools import GetLangChainPythonRepl
# from langchain.prompts import ChatPromptTemplate
# from langchain.tools.render import render_text_description_and_args
# from langchain.schema.runnable import RunnablePassthrough
# from langchain.agents.output_parsers.json import JSONAgentOutputParser
# from utils.custom_output_parsers.agent_intermediatte_steps import format_intersteps_to_compressed_scratchpad

# config = Configurations(provider='AzureChatOpenAI')
# search_tool = get_web_search_tools(config)
# weather_tool = GetFromOpenWeatherMap()
# time_tool = GetFromDatetimeModule()
# python_repl_tool = GetLangChainPythonRepl()
# tools = [search_tool, weather_tool, time_tool, python_repl_tool]
# tool_dict = {tool.name:tool for tool in tools}

# obj = hub.pull("jet-taekyo-lee/rag-style-react")
# agent_prompt = ChatPromptTemplate.from_messages([
#     obj.messages[0],
#     # MessagesPlaceholder(variable_name="chat_history"),  
#     obj.messages[1],
# ])

# agent_prompt = agent_prompt.partial(
#         tools=render_text_description_and_args(tools),
#         tool_names=", ".join([t.name for t in tools]),
# )



# agent = ( RunnablePassthrough.assign(compressed_agent_scratchpad = lambda x: format_intersteps_to_compressed_scratchpad(x["intermediate_steps"], agent_scratchpad_compression_chain))
# | agent_prompt 
# | llm
# | JSONAgentOutputParser() )

# agent_action = agent.invoke({'intermediate_steps':[], 'input':'Find out the current time in New York City and then provide the weather forecast for the next 3 days in the same location.'})
# observation = tool_dict[agent_action.tool].run(tool_input=agent_action.tool_input)

agent_action_log = '''Thought: The user wants to know the current time in New York City and the weather forecast for the next 3 days. I need to use the GetFromDatetimeModule tool to get the current time and the GetFromOpenWeatherMap tool to get the weather forecast. The current time is necessary to provide an accurate response. 

Action:
```
{
"action": "GetFromDatetimeModule",
"action_input": {"IANA_timezone": "America/New_York"}
}
```'''
observation = 'timezone: America/New_York, current time(year-month-day-hour-minute): 2023-11-12-7-18'
print(agent_action_log)
print(observation)

Thought: The user wants to know the current time in New York City and the weather forecast for the next 3 days. I need to use the GetFromDatetimeModule tool to get the current time and the GetFromOpenWeatherMap tool to get the weather forecast. The current time is necessary to provide an accurate response. 

Action:
```
{
"action": "GetFromDatetimeModule",
"action_input": {"IANA_timezone": "America/New_York"}
}
```
timezone: America/New_York, current time(year-month-day-hour-minute): 2023-11-12-7-18


### Get a response

In [30]:
comprerssed_agent_scratchpad = agent_scratchpad_compression_chain.invoke({'log':agent_action_log, 'observation':observation})
print(comprerssed_agent_scratchpad.content)

"The current time in New York City is 7:18 on November 12th, 2023."
